Good Links:

https://towardsdatascience.com/item-based-collaborative-filtering-in-python-91f747200fab

# Imports

In [ ]:
# Library
import pandas as pd
import numpy as n
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import *
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Locations
ml_100k = '/content/drive/My Drive/movie_recs/ml-100k'
ml_20m = '/content/drive/My Drive/movie_recs/ml-20m'


<h1>Random Stats</h1>

In [ ]:
column_names_a = ['genre, number']
genre = pd.read_csv(ml_100k +'/u.genre', sep='|', names=column_names_a)
# Checking shape of users files and head 
print(genre.shape)
genre


(19, 1)


,"genre, number"
unknown,0
Action,1
Adventure,2
Animation,3
Children's,4
Comedy,5
Crime,6
Documentary,7
Drama,8
Fantasy,9


<h1>Reading User Ratings</h1>

In [ ]:
column_names_b = ['user_id', 'item_id', 'rating', 'timestamp']
users= pd.read_csv(ml_100k +'/u.data', sep='\t', names=column_names_b)
# Checking shape of users files and head 
print(users.shape)
users.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Reading Movie

In [ ]:
# Reading the movie data
movies= pd.read_csv(ml_20m+'/movies.csv')
movies = movies.rename(columns = {'movieId':'item_id'})
# Checking shape of movie data and look first 5 rows
print(movies.shape)
movies.head()

(27278, 3)


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


<h1>FIND SIMILAR USERS</h1>

User ratings of all movies they rated

In [ ]:
df = pd.merge(users,movies,  on = 'item_id')
MoviesNUsers = df.pivot_table(index="user_id",columns='title',values='rating').fillna(0)
print(MoviesNUsers.shape)
MoviesNUsers.head()

(943, 1626)


title,'Til There Was You (1997),1-900 (06) (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (One Eight Seven) (1997),2 Days in the Valley (1996),2 ou 3 choses que je sais d'elle (2 or 3 Things I Know About Her) (1967),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),"301, 302 (301/302) (1995)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),"Yes, Madam (a.k.a. Police Assassins) (a.k.a. In the Line of Duty 2) (Huang gu shi jie) (1985)",Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Cosine Similarity

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  return cos_sim

#Just to test things out

a = MoviesNUsers.iloc[0] #user id = 1
b = MoviesNUsers.iloc[915] #user id = 916

cosine_similarity(a,b)

0.5674233765435364

Try make algorithm that tries to recommend movie for a specific user, say, top 5 ish movies

In [ ]:
def findSimilar(user_data, data, k): #Return top k users
  sim_of_user = []
  for i in range(len(data)):
      x = [data.iloc[i],cosine_similarity(user_data, data.iloc[i])]
      sim_of_user.append(x)
  sim_of_user.sort(key=lambda x: x[1], reverse = True)
  if sim_of_user[0][1] >= 1: #Delete the same users if they exist
    del sim_of_user[0]
  #del sim_of_user[0] #This is for when the user is in the same dataset, we delete the similarity to themselves, cos itll always be 1
  top_users = []
  for i in range(k):
    top_users.append(sim_of_user[i])
  return top_users

#Test, The numbers on the left refer to the iloc of the user, its a direct copy of what to put in the system
findSimilar(MoviesNUsers.iloc[1], MoviesNUsers, 3)


[[title
  'Til There Was You (1997)                     0.0
  1-900 (06) (1994)                             0.0
  101 Dalmatians (1996)                         0.0
  12 Angry Men (1957)                           0.0
  187 (One Eight Seven) (1997)                  0.0
                                               ... 
  Young Guns (1988)                             0.0
  Young Guns II (1990)                          0.0
  Young Poisoner's Handbook, The (1995)         0.0
  Zero Kelvin (Kjærlighetens kjøtere) (1995)    0.0
  Zeus and Roxanne (1997)                       0.0
  Name: 701, Length: 1626, dtype: float64, 0.5850179393017045], [title
  'Til There Was You (1997)                     0.0
  1-900 (06) (1994)                             0.0
  101 Dalmatians (1996)                         0.0
  12 Angry Men (1957)                           0.0
  187 (One Eight Seven) (1997)                  0.0
                                               ... 
  Young Guns (1988)                  

Make algrithm to predict the rating of movies

In [ ]:
cosine_similarity(MoviesNUsers.iloc[1],MoviesNUsers.iloc[700])

0.5850179393017045

Recommend movies from the top 5 similar people, by showing all movies that designated user has not seen, but candidate users have seen (without repeating the same movie of course).

In [ ]:
MoviesNUsers.iloc[1]

title
'Til There Was You (1997)                     0.0
1-900 (06) (1994)                             0.0
101 Dalmatians (1996)                         0.0
12 Angry Men (1957)                           0.0
187 (One Eight Seven) (1997)                  0.0
                                             ... 
Young Guns (1988)                             0.0
Young Guns II (1990)                          0.0
Young Poisoner's Handbook, The (1995)         0.0
Zero Kelvin (Kjærlighetens kjøtere) (1995)    0.0
Zeus and Roxanne (1997)                       0.0
Name: 2, Length: 1626, dtype: float64

In [ ]:
# Here we are going to cosine simalirity 
#import numpy as np
#from sklearn.neighbors import NearestNeighbors

# Building the model
#model_knn = NearestNeighbors(metric= 'manhattan', algorithm='brute')

# Fitting the model 
#model_knn.fit(MoviesNUsers)

# Data Pre-processing

Eliminate users with less than X amount of movies rated. We will see how much data gets removed

In [ ]:
import numpy as np
#RUN THIS AT YOUR OWN RISK, IT TAKES A WHILE TO COMPILE, BUT IT WORKS
'''X = 19 #Lucky for us, all users rated at LEAST 20 movies.
NewSet = MoviesNUsers
print(np.shape(NewSet))
for i in range(NewSet.shape[0]):
  count = 0
  for j in range(NewSet.shape[1]):
    if MoviesNUsers.iloc[i][j] > 0:
        count = count + 1
  if count < X:
    NewSet = NewSet.drop(i)
    print('dropped row:', i, np.shape(NewSet), end=" ")'''


'X = 19 #Lucky for us, all users rated at LEAST 20 movies.\nNewSet = MoviesNUsers\nprint(np.shape(NewSet))\nfor i in range(NewSet.shape[0]):\n  count = 0\n  for j in range(NewSet.shape[1]):\n    if MoviesNUsers.iloc[i][j] > 0:\n        count = count + 1\n  if count < X:\n    NewSet = NewSet.drop(i)\n    print(\'dropped row:\', i, np.shape(NewSet), end=" ")'

Perform Partitioning

In [ ]:
#CHOOSE HOWEVER YOU WANT TO PARTITION
NewSet = MoviesNUsers
Test = NewSet.sample(frac=0.2)
Train = NewSet.drop(Test.index)

print(np.shape(Test))
print(np.shape(Train))

(189, 1626)
(754, 1626)


Ommit data based on latest user interaction. First get timestamps

In [ ]:
TS_column_names = ['user_id', 'item_id', 'rating', 'timestamp']
users= pd.read_csv(ml_100k +'/u.data', sep='\t', names=TS_column_names)
# Checking shape of users files and head 
print(users.shape)
users.head()


(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Get movie based on item_id by combining columns

In [ ]:
movies= pd.read_csv(ml_20m+'/movies.csv')
movies = movies.rename(columns = {'movieId':'item_id'})
# Checking shape of movie data and look first 5 rows



In [ ]:
timed_ratings = pd.merge(users, movies, on= 'item_id')

In [ ]:
Train


title,'Til There Was You (1997),1-900 (06) (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (One Eight Seven) (1997),2 Days in the Valley (1996),2 ou 3 choses que je sais d'elle (2 or 3 Things I Know About Her) (1967),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),"301, 302 (301/302) (1995)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),"Yes, Madam (a.k.a. Police Assassins) (a.k.a. In the Line of Duty 2) (Huang gu shi jie) (1985)",Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Take out train set from time list

In [ ]:

for i in range (len(Train)):
  timed_ratings = timed_ratings.drop(timed_ratings.index[timed_ratings['user_id'] == int(Train.index[i])])# drop user id in training set
  print(len(timed_ratings), end = ' ')
timed_ratings


98409 98347 98294 98270 98095 97885 97489 97432 97253 96629 96531 96428 96288 96262 95988 95968 95921 95747 95619 95470 95392 95349 95314 95273 95250 95230 95205 95149 95028 95006 94971 94919 94737 94520 94370 94322 94295 94272 94207 93997 93974 93918 93890 93825 93642 93537 93384 93008 92988 92758 92666 92467 92387 92349 92319 92285 92221 92084 92018 91979 91902 91822 91751 91731 91677 91648 91590 91423 91269 91201 91178 90972 90899 90607 90221 89827 89552 89497 89434 89407 89350 89283 89254 89143 89120 89056 89024 88895 88871 88827 88777 88730 88638 88497 88319 88293 88233 88179 87999 87955 87932 87751 87402 87376 87352 87317 87270 87247 87226 87194 87175 87147 87127 87062 87028 86724 86619 86597 86576 86526 86406 86348 86306 86283 86222 86193 86174 86105 86037 86000 85972 85945 85904 85867 85804 85535 85493 85067 85039 84986 84739 84648 84592 84480 84297 84238 84211 84176 83872 83833 83717 83537 83499 83117 83097 83055 83013 82991 82928 82700 82667 82634 82502 82465 82339 82212 8211

,user_id,item_id,rating,timestamp,title,genres
3,154,242,3,879138235,Farinelli: il castrato (1994),Drama|Musical
26,145,242,5,875269755,Farinelli: il castrato (1994),Drama|Musical
29,144,242,4,888103444,Farinelli: il castrato (1994),Drama|Musical
34,12,242,5,879960826,Farinelli: il castrato (1994),Drama|Musical
36,131,242,5,883681723,Farinelli: il castrato (1994),Drama|Musical
...,...,...,...,...,...,...
98650,782,1668,3,891500067,One Night Stand (1997),Drama
98659,782,1665,2,891500194,Bean (1997),Comedy
98661,782,1670,3,891497793,Welcome to Sarajevo (1997),Drama|War
98666,782,1667,3,891500110,Mad City (1997),Action|Drama


Creating Target List

In [ ]:
target_list = []
timed_ratings[timed_ratings.timestamp == timed_ratings.timestamp.max()].iloc[0] #grab the max timestamp
while len(timed_ratings) != 0:
  target_list.append(timed_ratings[timed_ratings.timestamp == timed_ratings.timestamp.max()].iloc[0]) # grabs the row, and appends it to target
  drop_user = timed_ratings[timed_ratings.timestamp == timed_ratings.timestamp.max()].iloc[0][0] #assign user to be dropped
  timed_ratings = timed_ratings.drop(timed_ratings.index[timed_ratings['user_id'] == int(drop_user)])#after that, drop all rows with the same user id
  print(len(timed_ratings), end = " ")

18233 18128 17646 17361 17012 16811 16600 16457 16323 15954 15860 15837 15609 15504 15476 15455 15243 14989 14945 14915 14866 14845 14671 14567 14543 14228 13998 13973 13941 13845 13755 13682 13576 13139 13103 13026 13003 12768 12746 12725 12654 12571 12547 12526 12506 12404 12359 12240 12219 12187 11952 11928 11822 11605 11229 11074 10963 10942 10830 10622 10468 10086 10041 10021 9946 9917 9783 9670 9634 9572 9368 9347 9318 9042 9003 8982 8876 8853 8797 8648 8626 8557 8531 8480 8460 8384 8298 8156 8132 8101 8000 7966 7928 7903 7874 7720 7689 7636 7608 7436 7389 7327 7207 7178 7052 6922 6762 6690 6295 6189 6031 6002 5983 5936 5914 5773 5731 5711 5591 5563 5481 5419 5317 5240 5207 5128 4816 4797 4684 4627 4606 4578 4538 4307 4256 4074 4051 3986 3816 3783 3732 3677 3411 3361 3326 3230 3188 2972 2856 2785 2723 2679 2636 2587 2548 2528 2508 2487 2460 2297 2273 2221 2103 2032 1981 1930 1899 1858 1837 1791 1746 1667 1604 1567 1486 1320 1115 1091 1038 980 917 680 608 537 497 429 229 183 0 

To Dataframe

In [ ]:
target = pd.DataFrame(target_list)#make the target list into a dataframe

In [ ]:
print(target.iloc[1]['title'])
print(target.iloc[1]['user_id'])
print(target.iloc[1]['rating'])

Sword in the Stone, The (1963)
159
2


Re-assigns target values in the test to 0

In [ ]:

testtest = Test # i dont wanna do that real Test thing, so just do this instead
#print(testtest.at[796, 'Last Action Hero (1993)'])
for i in range(len(target)):
  testtest.at[target.iloc[i].user_id, target.iloc[i].title]=0 #update value at particular column to 0, this is what we will predict
  
#print(testtest.at[796, 'Last Action Hero (1993)']) #wanna see the values change, this changes everytime you run through the dataset

Sort Datasets for easier reading

In [ ]:
testtest = testtest.sort_values(by=['user_id'])

In [ ]:
target = target.sort_values(by=['user_id'])

# Testing/Benchmarking Cosine Distance

Cosine Similarity prediction

In [ ]:
def findSimilar_no_drop(user_data, data, k): #Return top k users
  sim_of_user = []
  for i in range(len(data)):
      x = [data.iloc[i],cosine_similarity(user_data, data.iloc[i])]
      sim_of_user.append(x)
  sim_of_user.sort(key=lambda x: x[1], reverse = True)
  if sim_of_user[0][1] >= 1: #Delete the same users if they exist
    del sim_of_user[0]
  #del sim_of_user[0] #This is for when the user is in the same dataset, we delete the similarity to themselves, cos itll always be 1
  top_20_plus_k = []
  for i in range(20 + k):
   top_20_plus_k.append(sim_of_user[i]) 
  return top_20_plus_k

findSimilar_no_drop(MoviesNUsers.iloc[1], MoviesNUsers)

TypeError: ignored

In [ ]:
def mass_cos_sim_predict(trainData, testData, target, k):
  predictions = []
  for i in range(len(testData)):
    sim_users = findSimilar(testData.iloc[i], trainData, k)
    #sim_users = findSimilar_no_drop(testData.iloc[i], trainData, k) 
    movie_to_predict = target.iloc[i]['title']#target and test set sorted, so its ok
    #movie_to_predict = target[target.user_id == i]['title']
    prediction = predict_rating(sim_users, movie_to_predict, k)
    comp = [prediction, target.iloc[i]['rating']]
    #comp = [prediction, target[target.user_id == i]['rating']]
    predictions.append(comp)
  return predictions
  
def evaluate_rmsd_cossim(predictions):
  #now check the error of the predicitons using RMSE/D
  sqsum = 0
  n = len(predictions)
  for i in range(len(predictions)):
    #if predictions[i][0] > 0: ##eliminate non zero entries
    sqsum = sqsum + (predictions[i][0] - predictions[i][1])**2
  rmsd = 1
  if n == 0:
      rmsd = 1
  else:
    rmsd = (sqsum/n)**(1/2)
  return rmsd



def predict_rating(sim_users, movie, k):
  numerator = 0
  denominator = 0
  predicted_rating = 0
  movie_index = list(sim_users[1][0].keys()).index(movie)
  #first check if the user has a rating rated the movie
  #if they did, add their sim score to denom, add their rating*score to numerator
  buffer = 0
  i = 0
  for i in range(len(sim_users)):
    #if sim_users[i][0][int(movie_index)] > 0:
    numerator = numerator + sim_users[i][0][movie_index]*sim_users[i][1] #numerator calc; similariy * rating
    denominator = denominator + sim_users[i][1] #sum of non missing similarity scores. non missing means that similar users have watched the target movie, otherwise, they dont cotnribute
  if denominator == 0:
    predicted_rating = 0#default for when there are no users, we can set this to a custom variable to increase performane (2.5)
  else:
    predicted_rating = numerator/denominator
    
  return predicted_rating
#workflow on on iterationL: for all entries in 

In [ ]:
predictions = mass_cos_sim_predict(Train, testtest, target, 5)
print(predictions)

[[0.0, 5], [1.1824875173366145, 4], [0.0, 5], [2.040561977262205, 4], [2.977177649064591, 4], [1.1689786937948399, 3], [0.7884710457653838, 2], [0.0, 4], [0.0, 5], [0.0, 4], [1.9858439774750318, 5], [0.817776730760587, 4], [0.5817469996979305, 1], [2.381410636615578, 5], [0.7973234179706404, 4], [0.38611270427459415, 5], [2.551802181369533, 4], [0.0, 2], [1.0072920649989745, 1], [0.614630060996153, 1], [0.5901469532905079, 3], [0.21829588156984037, 2], [0.0, 2], [0.38213038911702624, 4], [0.0, 5], [2.688881126068442, 3], [1.1983010941328343, 1], [1.7656173094758298, 5], [0.0, 4], [0.0, 3], [3.462660544850854, 5], [0.0, 1], [2.244825773019618, 4], [0.0, 1], [0.583385209518208, 1], [2.4135970097284214, 3], [3.642980686455202, 3], [1.6490588471045164, 4], [0.1888971137204608, 2], [0.5454153849912495, 5], [0.9122636499647602, 1], [1.0083073190778706, 2], [3.402766613271058, 5], [0.0, 5], [2.1934812865270152, 4], [3.6132364935697154, 4], [0.4147862572018203, 3], [2.8788669189432703, 4], [0.

In [ ]:
evaluate_rmsd_cossim(predictions)

2.6417824449746354